## Approach
Use SIFT to obtain image embeddings each bird and store them in FAISS; then see if we can correctly find bird species given an image.

To start: Install packages by running `pip install pytorch opencv-contrib-python` in a terminal window `conda install -c pytorch faiss-gpu`.
OReilley's [Practical Deep Learning Book](https://www.oreilly.com/library/view/practical-deep-learning/9781492034858/ch04.html) chapter 4 gives us some guidance here also.

In [ ]:
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

matplotlib.use('Agg')
%matplotlib inline

import pickle
import random
import os
import time

from tqdm import tqdm, tqdm_notebook

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE

import faiss

import numpy as np
from numpy.linalg import norm

import keras
from keras.preprocessing import image


### Produce Embeddings of All our Images

In [ ]:
import tensorflow_hub as hub

# load model to produce our embeddings
model = hub.KerasLayer("https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5", trainable=False) # targetsize 299 for this

def extract_features(img_path, model, target_size=(299, 299)):
    """Load image to arary, resize, scale and expand dimensions."""
    img = image.load_img(img_path, target_size = target_size)
    img = image.img_to_array(img)
    
    # scale to [|0, 1]
    img = img / 255
    
    # expand dim
    img = np.expand_dims(img, axis = 0)
    
    # get embeddings for and flatten
    img_embeddings = model(img)
    img_embeddings = img_embeddings.numpy()[0]
        
    # normalise images
    img_embeddings /= norm(img_embeddings)

    return img_embeddings

In [ ]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                img_path = os.path.join(root, filename)
                if '.ipynb' in img_path: continue
                file_list.append(img_path)
                counter += 1
    return file_list

In [ ]:
# path to the datasets; do birds as it is smaller subset
root_dir = './data/train_mini_supercategory/Animalia' 
filenames = sorted(get_file_list(root_dir))
num_images = len(filenames)
print(num_images, 'files found')

In [ ]:
# define variable to store all our features
feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))

In [ ]:
# pickle our files
# pickle.dump(feature_list, open('./models/features-inceptionv3.pickle', 'wb'))
# pickle.dump(filenames, open('./models/filenames.pickle','wb'))

MobileNet is much faster than ResNet for inference.

### View These Images
Helper functions found [here](https://blog.csdn.net/guaguastd/article/details/107777972)

In [ ]:
# # load features list and filenames
with open('./models/features-inceptionv3.pickle', 'rb') as pickle_file:
    feature_list = pickle.load(pickle_file)
feature_list = np.array(feature_list)
    
with open('./models/filenames.pickle', 'rb') as pickle_file:
    filenames = pickle.load(pickle_file)
num_images = len(filenames)

In [ ]:
def classname_filename(str):
    return str.split('/')[-2]

def plot_images(filenames, distances):
    images = []
    for filename in filenames:
        images.append(mpimg.imread(filename))
    plt.figure(figsize=(20, 10))
    columns = 4
    for i, image in enumerate(images):
        ax = plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        if i==0:
            ax.set_title("Query Image \n" + classname_filename(filenames[i]))
        else:
            ax.set_title("Similar Image\n" + classname_filename(filenames[i]) + "\nDistance: " + f"{distances[i]:.2f}")
        plt.imshow(image)

In [ ]:
# get nearest neighbours and plot them
neighbors = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list)

for i in range(3):
    random_image_index = random.randint(0,num_images-1)
    distances, indices = neighbors.kneighbors([feature_list[random_image_index]])
    # don't take the first closest image as it will be the same image
    similar_image_paths = [filenames[random_image_index]] +[filenames[indices[0][i]] for i in range(1,4)]
    plot_images(similar_image_paths, distances[0])

In [ ]:
def get_top_1_topn(indexes, n=5):

    times_top1 = 0
    times_top5 = 0

    for i in tqdm(I):

        actual_name = classname_filename(filenames[i[0]])

        if actual_name == classname_filename(filenames[i[1]]):
            times_top1 += 1
            times_top5 += 1
            continue

        if actual_name in [classname_filename(filenames[ii]) for ii in i[1:]]:
            times_top5 += 1
            continue
            
    return times_top1, times_top5

In [ ]:
%%timeit -r 1 -n 1
# bruteforce; figure out times top1, top5. Really slow though, definitely be going with faiss
all_indices=[]
for i in tqdm(range(0,7100)):
    distances, indices = neighbors.kneighbors([feature_list[i]])
    all_indices.append(indices)
all_indices = np.array(all_indices)

times_top1, times_top5 = get_top_1_topn(all_indices, 5)
print('Times top1:', times_top1, '\nTimes top5:', times_top5)

In [ ]:
# Perform PCA over the features
num_feature_dimensions = 50      # Set the number of features
pca = PCA(n_components = num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)

# For speed and clarity, we'll look at first 3000/~7000 images
num_samples = 3000
selected_features = feature_list_compressed[:num_samples]
selected_filenames = filenames[:num_samples]
# get classids and map them to colors to plot
selected_class_ids = filenames[:num_samples]
selected_class_ids = [scid.split('/')[-2] for scid in selected_class_ids if scid != '.ipynb_checkpoints']
idx_classid_map = {cid:idx for idx, cid in enumerate(set(selected_class_ids))}
class_id_colors = [idx_classid_map[cid] for cid in selected_class_ids]

# get tsne results
tsne_results = TSNE(n_components=2, verbose=1,metric='euclidean').fit_transform(selected_features)

# Plot a scatter plot from the generated t-SNE results
colormap = plt.cm.get_cmap('coolwarm')
scatter_plot = plt.scatter(tsne_results[:,0],tsne_results[:,1], c = class_id_colors, cmap=colormap)
plt.colorbar(scatter_plot)
plt.show();

In [ ]:
from PIL import Image
import PIL
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)

def plot_images_in_2d(x, y, image_paths, axis=None, zoom=1):
    if axis is None:
        axis = plt.gca()
    x, y = np.atleast_1d(x, y)
    for x0, y0, image_path in zip(x, y, image_paths):
        image = Image.open(image_path)
        image.thumbnail((100, 100), Image.ANTIALIAS)
        img = OffsetImage(image, zoom=zoom)
        anno_box = AnnotationBbox(img, (x0, y0), xycoords='data', frameon=False)
        axis.add_artist(anno_box)
    axis.update_datalim(np.column_stack([x, y]))
    axis.autoscale()

In [ ]:
plt.figure(figsize=(45,45))
plot_images_in_2d(tsne_results[:,0], tsne_results[:,1], selected_filenames)
# plt.savefig('inceptionv3-normalized.png')
plt.show()

### PCA To Reduce Dimensionality
[Link to Git](https://github.com/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-4/3-reduce-feature-length-with-pca.ipynb) of O'Reilley book

In [ ]:
num_feature_dimensions=200 # reduce to 200 dimensions
pca = PCA(n_components = num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)

In [ ]:
# visualise importance of first 200 dimensions

matplotlib.style.use('seaborn')
plt.plot(range(1,201),pca.explained_variance_ratio_,'o--', markersize=4)
plt.title ('Variance for each PCA dimension')
plt.xlabel('PCA Dimensions')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(range(1,201),pca.explained_variance_ratio_.cumsum(),'o--', markersize=4)
plt.title ('Cumulative Variance with each PCA dimension')
plt.xlabel('PCA Dimensions')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
# Helper function to get the classname
def classname(str):
    return str.split('/')[-2]

# Helper function that calculates accuracy using the nearest neighbors brute force algorithm
def calculate_accuracy(feature_list):
    num_nearest_neighbors = 5
    correct_prediction = 0
    incorrect_prediction = 0
    neighbors = NearestNeighbors(n_neighbors=num_nearest_neighbors,
                                 algorithm='brute',
                                 metric='euclidean').fit(feature_list)
    start = time.time()
    for i in range(len(feature_list)):
        distances, indices = neighbors.kneighbors([feature_list[i]])
        for j in range(1, num_nearest_neighbors):
            if (classname(filenames[i]) == classname(
                    filenames[indices[0][j]])):
                correct_prediction += 1
            else:
                incorrect_prediction += 1
    end = time.time()
    accuracy = round(
        100.0 * correct_prediction /
        (1.0 * correct_prediction + incorrect_prediction), 2), end - start
    return accuracy

In [ ]:
pca_dimensions = [1,2,3,4,5,10,20,50,75,100,150,200]
pca_accuracy = []
pca_time = []

for dimensions in pca_dimensions:
    # Perform PCA
    pca = PCA(n_components = dimensions)
    pca.fit(feature_list)
    feature_list_compressed = pca.transform(feature_list[:])
    # Calculate accuracy over the compressed features
    accuracy, time_taken = calculate_accuracy(feature_list_compressed[:])
    pca_time.append(time_taken)
    pca_accuracy.append(accuracy)
    print("For PCA Dimensions = ", dimensions, ",\tAccuracy = ",accuracy,"%", ",\tTime = ", pca_time[-1])

In [ ]:
plt.plot(pca_time, pca_accuracy,'o--', markersize=4)
for label, x, y in zip(pca_dimensions, pca_time,pca_accuracy):
    plt.annotate(label, xy=(x, y), ha='right', va='bottom')
plt.title ('Test Time vs Accuracy for each PCA dimension')
plt.xlabel('Test Time')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

Decide to go with 100 features, as the benefits flatten for the amount of increased test time. <br>
The dimensions should correspond with the `filenames` variable.

In [ ]:
feature_list.shape

### FAISS For ANN Search (with GPU's)
Another Option is Spotify's [annoy](https://github.com/spotify/annoy)

In [ ]:
num_feature_dimensions=100 # reduce to 100 dimensions
pca = PCA(n_components = num_feature_dimensions)

pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
assert feature_list_compressed.shape == (num_images, num_feature_dimensions)

feature_list_compressed = feature_list_compressed.astype(np.float32)

In [ ]:
feature_list = np.array(feature_list).astype(np.float32)
feature_list.shape

In [ ]:
INDEX_KEY = "Flat"
use_gpu = False # false in this notebook

In [ ]:
# build faiss index
index = faiss.index_factory(2048, INDEX_KEY)

if use_gpu:
    print('Using GPU')
    # if this fails, it means that the GPU version was not comp
    assert faiss.StandardGpuResources, \
        "FAISS was not compiled with GPU support, or loading _swigfaiss_gpu.so failed"
    res = faiss.StandardGpuResources()
    dev_no = 0

    # transfer to GPU (may be partial)
    index = faiss.index_cpu_to_gpu(res, dev_no, index)
    params = faiss.GpuParameterSpace()

print(index.is_trained)

In [ ]:
%%timeit -r 1 -n 1 # timeit
filenames[:5]

In [ ]:
feature_list_compressed.shape

In [ ]:
%%timeit -r 1 -n 1
# add indexes
#index.add(feature_list_compressed)
index.add(feature_list)

In [ ]:
%%timeit -r 1 -n 1
D, I = index.search(feature_list[:5], 5) # sanity check, 5 Nearest-Neighbours
print(I) # indexes
print(D) # distances

In [ ]:
D, I = index.search(feature_list[:5], 5) # sanity check, 5 Nearest-Neighbours

In [ ]:
I[:5]

In [ ]:
# similar_image_paths = [filenames[random_image_index]] +[filenames[indices[0][i]] for i in range(1,4)]
# plot_images(similar_image_paths, distances[0])

for i, d in zip(I, D):
    similar_image_paths = [filenames[ii] for ii in i]
    plot_images(similar_image_paths, d)


In [ ]:
D, I = index.search(feature_list[:7000], 5) # sanity check

In [ ]:
%%timeit -r 1 -n 1
# of n images, we will count the amount that match top-1, top 5
D, I = index.search(feature_list[:7000], 5) 

In [ ]:
times_top1, times_top5 = get_top_1_topn(I, 5)
print('Times top1:', times_top1, '\nTimes top5:', times_top5)

In [ ]:
#faiss.write_index(faiss.index_gpu_to_cpu(gpu_index), writer.data) # if gpu
faiss.write_index(index, './models/faiss.index')

In [ ]:
index = faiss.read_index('./models/faiss.index')